In [59]:
EXPERIMENT_PATH = "../experiments"
OUTPUT_PATH = "../output"
SEVERITY_GROUP_DICT = {
    'mild':['mild'],
    'moderate':['moderate','mod', 'moder','moderat'],
    'sever':['sever','sev', 'severe','seve'],
    'TD':['td'],
}

TEMP_EXPERIMENT = {
    "data_repr": "percentile",
    "normalizer": "minmax",
    "DD": {
        "srs_type": "cog",
        "severity_group": ["mild", "TD"],
        "age_group": None,
        "divide_data": False,
        "balance": False
    },
    "FS": {
        "est": ["lsvm", "rf", "xgb", "lr"],
        "cv": 5,
        "scoring": "balanced_accuracy",
        "n_jobs": -1,
        "verbose": 0,
        "step": 1,
        "min_features_to_select": 1
    },
    "ML": {
        "est": ["lr", "lsvm", "svm", "xgb", "rf", "ridge", "gnb","nn"],
        "cv": 5,
        "scoring": "balanced_accuracy",
        "n_jobs": -1,
        "verbose": 0,
        "hyper_search_type": "random",
        "n_iter": 1500}
}

In [2]:
import json
import os

# Goals
Utilizing the TEMP_EXPERIMENT to build different experiments combinations and place them in a hierarchial folder structer that 
defined every experiement with a name

# Define The experiments I want to build

In [4]:
severity_comparisons = [('sever', 'td'),('mild','td'),('moderate','td')]
age_groups = [(5,8),(9,12),(13,15),(16,18),(19,21),(22, 28), (28, None)]

In [52]:
# Define the nested comparison structure
comparisons = {
    ("DD__severity_group",('sever', 'td')):list(map(lambda x: ("DD__age_group",)+((x)), age_groups)),
    ("DD__severity_group",('mild', 'td')): list(map(lambda x: ("DD__age_group",)+((x)), age_groups)),
    ("DD__severity_group",('moderate', 'td')):list(map(lambda x:("DD__age_group",)+((x)), age_groups))
    
}

In [53]:
def create_directory(path):
    if not os.path.isdir(path):
        os.mkdir(path)
    return path

In [60]:
def adjust_TEMP_EXPERIMENT(temp_exp, keys_list, values_list):
    for key, value in zip(keys_list, values_list):
        key_parts = key.split('__')
        if len(key_parts)>1:
            pkey, skey = key_parts[0], key_parts[-1]
            temp_exp[pkey][skey] = value
        else:
            print(key)
            temp_exp[key] = value
            
    return temp_exp

In [63]:
sub_fldr_str = f"Agebetween!t*_^^_{TEMP_EXPERIMENT['data_repr']}_{TEMP_EXPERIMENT['normalizer']}"
json_file_str = f"{TEMP_EXPERIMENT['data_repr']}_{TEMP_EXPERIMENT['normalizer']}_^^_$$.json"
for main_group, factor_groups in comparisons.items():
    # Creating the parent paths
    main_key = main_group[0]
    main_comparison = main_group[1]
    path = create_directory(os.path.join(EXPERIMENT_PATH,f'{"_".join([str(x) for x in main_comparison])}'))
    # Creating the subpaths
    for factor in factor_groups:
        fkey = factor[0]
        groups = factor[1:]
        subfldr = sub_fldr_str.replace('!', str(groups[0])).\
                  replace("*", str(groups[1])).\
                  replace("^^", "_".join([str(x) for x in main_comparison]))
        subpath = create_directory(os.path.join(path,subfldr))
        for srs_test in ["awa","cog","comm","man","mot","tot"]:
            srs_json = json_file_str.replace("^^","_".join([str(x) for x in main_comparison])).\
                                            replace("$$",srs_test)
            full_srs_json = os.path.join(subpath, srs_json)
            exp_dict = adjust_TEMP_EXPERIMENT(TEMP_EXPERIMENT,
                                              [main_key, fkey, f"DD__srs_type"],
                                              [main_comparison, groups, srs_test])
            with open(full_srs_json, 'w') as f:
            json.dump(exp_dict, f)